# Processing data for modelling

## - Stock price

In [1]:
import pandas as pd

from tqdm import tqdm





In [2]:
# test with feature_engineering.py
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
print(prices.shape)
prices.head(2)

(2332531, 12)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324


In [3]:

from feature_engineering import fill_and_drop_na_values

fill_prices = fill_and_drop_na_values(prices)
print(fill_prices.shape)
fill_prices.head(2)


100%|██████████| 2000/2000 [09:08<00:00,  3.65it/s]


(2332531, 12)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.00073
1865,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.00292


In [4]:
from feature_engineering import adjust_price

adjusted_price = adjust_price(fill_prices)
print(adjusted_price.shape)
adjusted_price.head(2)

100%|██████████| 2000/2000 [1:29:53<00:00,  2.70s/it]    


(2332531, 18)


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,ad_Open,ad_High,ad_Low,ad_Close,ad_Volume,ad_Target
1483,20170104_8194,2017-01-04,8194,3325.0,3390.0,3300.0,3380.0,36100,1.0,NaN,False,-0.002963,3325.0,3390.0,3300.0,3380.0,36100.0,-0.002963
3348,20170105_8194,2017-01-05,8194,3395.0,3420.0,3360.0,3375.0,19400,1.0,NaN,False,-0.020802,3395.0,3420.0,3360.0,3375.0,19400.0,-0.020802


In [5]:
from functions import df_security_code

df = df_security_code(adjusted_price, 8194)
df.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,ad_Open,ad_High,ad_Low,ad_Close,ad_Volume,ad_Target
1483,20170104_8194,2017-01-04,8194,3325.0,3390.0,3300.0,3380.0,36100,1.0,NaN,False,-0.002963,3325.0,3390.0,3300.0,3380.0,36100.0,-0.002963
3348,20170105_8194,2017-01-05,8194,3395.0,3420.0,3360.0,3375.0,19400,1.0,NaN,False,-0.020802,3395.0,3420.0,3360.0,3375.0,19400.0,-0.020802
5213,20170106_8194,2017-01-06,8194,3365.0,3385.0,3335.0,3365.0,23000,1.0,NaN,False,-0.00607,3365.0,3385.0,3335.0,3365.0,23000.0,-0.00607
7078,20170110_8194,2017-01-10,8194,3350.0,3360.0,3290.0,3295.0,42400,1.0,NaN,False,-0.022901,3350.0,3360.0,3290.0,3295.0,42400.0,-0.022901
8943,20170111_8194,2017-01-11,8194,3330.0,3330.0,3260.0,3275.0,20200,1.0,NaN,False,0.054688,3330.0,3330.0,3260.0,3275.0,20200.0,0.054688


NameError: name 'rsi' is not defined

In [ ]:
# TODO  features RSI

def price_features(price):
    
    def rsi (df_serie,periods = 14, ema = True): # x = 5,7,9,14,21,30
        # RSI = 100–(100/1 + RS) ; RS = Av Gain / Av loss

        
        close_delta = df_serie.diff() # .dropna()

        # Make two series: one for lower closes and one for higher closes
        up = close_delta.clip(lower=0)
        down = -1 * close_delta.clip(upper=0)

        if ema == True: # exponential moving average
            ma_up = up.ewm(com= periods - 1, adjust=True, min_periods = periods).mean()
            ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
        else: # using moving average
            ma_up = up.rolling(window = periods, adjust=False).mean()
            ma_down = down.ewm(window = periods, adjust=False).mean()
        
        rs = ma_up / ma_down
        rsi = 100 - (100 / (1+rs))

        return rsi

    # for each security
    codes = set(price.SecuritiesCode.unique())

    # create an empty new DataFrame
    price_features = pd.DataFrame(columns = price.columns)

    for i in tqdm(codes):
        df = price.query('SecuritiesCode ==@i')
        df['rsi'] = rsi(df['ad_Close'], 14)  # Werte über 70 als überkauft (geht runter) und Werte unter 30 als überverkauft
        
        

        #adjusted_df = adjust_price_slide(df)
        price_features = pd.concat([price_features, df], axis=0)
    price_features['Date'] = pd.to_datetime(price_features['Date']) 

    return price_features


## - Financials

In [ ]:
#from feature_engineering import fill_finances

financial = financial = pd.read_csv('data/train_files/financials.csv',parse_dates=[2])
financial.head(2)

In [ ]:
from feature_engineering import fill_finances_knn



filled_finances = fill_finances_knn(financial, prices)
filled_finances.head()
filled_finances.shape

In [ ]:
# ffill and drop 


def new_features_financial(filled_finances):
    sec_codes = filled_finances.SecuritiesCode.unique()

    filled_financial_feat = pd.DataFrame(columns=filled_finances.columns)

    for i in tqdm(sec_codes):
        # select a security code
        aktie = filled_finances.query('SecuritiesCode == @i')
        aktie.sort_values('Date')
        # create new features:
        aktie['margin'] = aktie['Profit'] / aktie['NetSales'] * 100
        aktie['profit_ttm'] = aktie['Profit'].shift(3) + aktie['Profit'].shift(2) + aktie['Profit'].shift(1) + aktie['Profit']
        aktie['rev_ttm'] = aktie['NetSales'].shift(3) + aktie['NetSales'].shift(2) + aktie['NetSales'].shift(1) + aktie['NetSales']
        aktie['win_quarter_growth'] = (aktie['Profit'] - aktie['Profit'].shift(1)) / aktie['Profit'].shift(1) * 100
        aktie['rev_quarter_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(1)) / aktie['NetSales'].shift(1) * 100
        aktie['win_yoy_growth'] = (aktie['Profit'] - aktie['Profit'].shift(4)) / aktie['Profit'].shift(4) * 100
        aktie['rev_yoy_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(4)) / aktie['NetSales'].shift(4) * 100
        aktie['win_ttm_growth'] = (aktie['profit_ttm'] - aktie['profit_ttm'].shift(1)) / aktie['profit_ttm'].shift(1) * 100
        aktie['rev_ttm_growth'] = (aktie['rev_ttm'] - aktie['rev_ttm'].shift(1)) / aktie['rev_ttm'].shift(1) * 100

        # fill
        aktie = aktie.ffill()
        aktie = aktie.dropna(axis=0)

        filled_financial_feat  = pd.concat([filled_financial_feat , aktie])

        filled_financial_feat['Date'] = pd.to_datetime(filled_financial_feat['Date']) 
    
    return filled_financial_feat

In [ ]:
filled_financial_features = new_features_financial(filled_finances)

In [ ]:
"""from functions import plot_stock
code = 1301

plot_stock(filled_financial_features, code,'win_ttm_growth' )"""

In [ ]:
filled_financial_features.head(2)


In [ ]:
filled_financial_features.SecuritiesCode = filled_financial_features.SecuritiesCode.astype(int)

In [ ]:
filled_financial_features.head(2)

In [ ]:
adjusted_price.head(2)

## - financials + price

In [ ]:
# create key on financial : RowId
filled_financial_features['RowId'] = filled_financial_features.Date.dt.strftime('%Y%m%d').astype(str) + '_' + filled_financial_features.SecuritiesCode.astype(str)

In [ ]:
#filled_financial_features.sort_values('Date').head(20)

In [ ]:
filled_financial_features.RowID.info()

In [ ]:
# concat financials and price
price_financial = pd.merge(adjusted_price, filled_financial_features, how='left', on='RowId', suffixes=[None, 'f_'])
#price_financial = pd.concat([adjusted_price, filled_financial_features], keys='RowID')

In [ ]:
price_financial.shape

In [ ]:
price_financial.head()

In [ ]:
price_financial.query('RowID == "20180209_1301"')


In [ ]:
price_financial.shape

In [ ]:
from functions import missingValues

missingValues(price_financial)

In [ ]:
# ffill and drop 

def fill_drop_pro_code(df):
    sec_codes = df.SecuritiesCode.unique()

    df_fill_na = pd.DataFrame(columns=price_financial.columns)

    for i in tqdm(sec_codes):

        current = df.query('SecuritiesCode == @i')
        current.sort_values('Date')
        current = current.ffill()
        current = current.dropna(axis=0)

        df_fill_na  = pd.concat([df_fill_na , current])

    df_fill_na['Date'] = pd.to_datetime(df_fill_na['Date'])
    
    return df_fill_na

In [ ]:
price_financial_fill = fill_drop_pro_code(price_financial)

In [ ]:
# new features
#price_financial_fill.drop_duplicates(inplace=True)
price_financial_fill.shape

In [ ]:
# save dataframe to csv
price_financial_fill.to_csv('data/current.csv')

TODOS: 

In [ ]:
# TODO encode flag